In [1]:
import cv2
import numpy as np
import json
import os
import os.path as osp
from pathlib import Path
from tqdm import tqdm
from time import time

from core import ObjectDetector, CornerPredictor

In [2]:
#dt_vehicle = ObjectDetector('yolov4-default')
#dt_plate   = ObjectDetector('yolov4-plate_type')
dt_number  = ObjectDetector('yolov4-plate_number')
#dt_corner  = CornerPredictor('corner_prediction')

<yolov4-plate_number> model loaded.


In [3]:
def start(img, pts):
    s_time = time()
    img_plate = dt_number.loadImage(img, True, pts)
    bbox3, obj3 = dt_number.runDetection(mode='number', multi_res=True)

    d_number = ''
    if len(bbox3) != 0:
        nb = [obj3[i] for i in bbox3[:, 0, 0].argsort()]
        d_number = ''
        for n in nb:
            d_number += n

    t_time = time() - s_time

    return t_time, d_number, img_plate

def error_log(p, label, predict, write=False, img=None):
    log = {'path': p, 'label': label, 'predict': predict}
    
    if write:
        dir = 'error_log/number_error_ourDataset_original'
        file = osp.basename(p).split('-')[0]
        if not osp.exists(dir): os.mkdir(dir)
        cv2.imwrite(osp.join(dir, file+'-plate.jpg'), cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    return log

In [4]:
dataset_dir = r'D:\User Profiles\Desktop\Personal Files\School\University Project\myPlateDetection\all_dataset\train_valid_data'
vehicle_dataset = osp.join(dataset_dir, 'vehicle_images')
plate_dataset = osp.join(dataset_dir, 'plate_images')

path_file = r'D:\User Profiles\Desktop\Personal Files\School\University Project\myPlateDetection\training_data\plate_number_detection\file_path.json'
with open(path_file, 'r') as f:
    test_path = json.load(f)['valid']

test_path = [osp.join(vehicle_dataset, osp.basename(p).split('-')[0]+'.jpg') for p in test_path]
print('total:', len(test_path))

total: 3496


In [5]:
n_test = len(test_path)
no_result = []
num_error = []
total_time = 0

acc_list = [0, 0]

for i in tqdm(range(n_test), unit=' images', ncols=100):
    p = test_path[i]
    img = cv2.cvtColor(cv2.imread(p), cv2.COLOR_BGR2RGB)
    with open(p.replace('jpg', 'json'), 'r', encoding='utf-8') as f:
        pts = np.array(json.load(f)['shapes'][0]['points']) # plate points

    if i == 0:
        _, _, _ = start(img, pts) # skip initial time
    t_time, d_number, img_plate = start(img, pts)

    fn = osp.basename(p).split('.')[0]
    number_file = osp.join(plate_dataset, fn+'-plate.json')
    with open(number_file, 'r') as f:
        nb = [sh['label'] for sh in json.load(f)['shapes']]  # plate numbers
    number = ''
    for n in nb:
        number += n

    total_time += t_time
    acc_list[0] += 1
    if d_number == number:
        acc_list[1] += 1
    elif d_number == '':
        no_result.append(error_log(fn, number, d_number, True, img_plate))
    else:
        num_error.append(error_log(fn, number, d_number, True, img_plate))

print('Average time: {:.3f}s'.format(total_time/n_test))
print('Overall: total={:>4d}, correct={:>4d}, accuracy={:.2f}%'.format(acc_list[0], acc_list[1], acc_list[1]/acc_list[0]*100))
print('No result:', len(no_result))
print('Number error:', len(num_error))

log_dict = {'no_result': no_result, 'num_error': num_error}
with open('error_log/number_error_ourDataset_original.json', 'w') as f:
    json.dump(log_dict, f, indent=2)

100%|██████████████████████████████████████████████████████| 3496/3496 [02:33<00:00, 22.82 images/s]

Average time: 0.035s
Overall: total=3496, correct=2694, accuracy=77.06%
No result: 146
Number error: 656
